In [ ]:
%matplotlib inline
%pylab inline
from beras.gan import GAN
from beras.models import AbstractModel
from beras.util import upsample, downsample, blur
import cairosvg
from deepdecoder.generate_grids import BlackWhiteArtist, MASK, MASK_BLACK, \
    MASK_WHITE, GridGenerator, MaskGridArtist
import deepdecoder.generate_grids as gen_grids

import h5py
import itertools
import keras
import keras.initializations
from keras.models import Sequential, Graph
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape, Layer
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSample2D

import os.path

import base64

import os
import xml.etree.ElementTree as et
import io
import scipy.misc
from deepdecoder import NUM_CELLS
import sys
from keras.optimizers import SGD, Adam, Adadelta
from scipy.misc import imsave
import numpy as np
from theano.ifelse import ifelse
import theano
import theano.tensor as T
import theano.tensor.shared_randomstreams as T_random
import time
import math
import matplotlib.pyplot as plt
from beras.layers.attention import RotationTransformer
from mask_loss import mask_loss 
from gan_train import *

pylab.rcParams['figure.figsize'] = (12, 6)
floatX = theano.config.floatX

batch_size = 128

In [ ]:
gan = MaskGAN(get_gan_dense_model())
weight_dir = "/home/leon/repos/keras_models/lapgan/models/gan_dense_and_mask_night_11_19_enforce"
if os.path.exists(weight_dir):
    print("Loading weights from: " + weight_dir)
    # gan.load(weight_dir)

In [ ]:
X = tags_from_hdf5("/home/leon/repos/deeplocalizer_data/data/lapgan/lapgan.txt")
X = X[:(len(X)//64)*64]

In [ ]:
def optimizer(name):
    if name in ["g", "inv", "d"]:
        return SGD()
    else:
        return Adam()
             
print("Compiling...")
start = time.time()
gan.compile(optimizer, functions=["debug", "train_gan_mask"], 
            gan_regularizer=lambda: GANL2Regularizer(low=1.18, high=1.3))
print("Done Compiling in {0:.2f}s".format(time.time() - start))

In [ ]:
try:
    for i, (masks_idx, mask_bits, mask_grids) in \
        enumerate(itertools.islice(masks(len(X), scales=[1]), 250)):
            
        print(i)
        gan.fit([X, masks_idx], direct_indexing=[False, True], verbose=1, nb_epoch=1, train_mode='gan_mask')
finally:
    weight_dir = "models/gan_dense_and_mask_night_11_19_base_sgd"
    gan.save(weight_dir)
    print("Save network to {}".format(weight_dir))

In [ ]:
weight_dir = "models/gan_dense_decoder_day_11_13_bad"
gan.save_weights(weight_dir + "/{}.hdf5")
with open(weight_dir + "/g.json", "w") as f:
    f.write(gan.gan.G.to_json())
with open(weight_dir + "/d.json", "w") as f:
    f.write(gan.gan.D.to_json())
with open(weight_dir + "/decoder.json", "w") as f:
    f.write(gan.decoder.to_json())

In [ ]:
import random
import operator
def draw_debug(mask_idx, mask_bits, outs, b=None):
    def bits_to_str(bits):
        s = ""
        for k in range(len(bits)):
            if bits[k] > 0.5:
                s += " "
            else:
                s += "#"
        return s
                      
    if b is None:
        b = random.randint(0, 63)
    i = 1
    print(b)
    n_plots = 7
    loss_img = np.zeros_like(mask_idx)
    for name, arr in sorted(outs.items(), key=operator.itemgetter(0)):
        if name == "d_out":
            print("fake: {0:.5f}".format(float(arr[b])))
            print("real: {0:.5f}".format(float(arr[b + batch_size // 2])))
        elif name == "mask_loss_per_sample":
            print("mask_loss: {0:.5f}".format(float(arr[b])))
        elif name == "cell_losses": 
            for j, key in enumerate(MASK_BLACK + MASK_WHITE):
                idx = mask_idx == MASK[key]
                # loss_img[idx] = arr[j, b]
        elif name in ["image", "mask_bw", "real", "decoder_rotation"]:
            plt.subplot(1, n_plots, i)
            i += 1
            plt.imshow(arr[b, 0], cmap='gray')           
            plt.colorbar(fraction=0.046, pad=0.04)
            plt.title(name)
        elif name in ["black_white_loss", "ring_loss"]:
            print("{}: {}".format(name, arr[b]))
        elif name == "predicted_labels":
            pass
            #print("predicted: " + bits_to_str(arr[b]))
            #print("true:      " + bits_to_str(mask_bits[b]))
        elif name == "g_invert_out":
            plt.subplot(1, n_plots, i)
            print(arr.shape)
            arr = arr.reshape(64, 2, 32, 32)
            i += 1
            plt.imshow(arr[b, 0], cmap='gray')           
            plt.colorbar(fraction=0.046, pad=0.04)
            plt.title(name + "_bw")
            plt.subplot(1, n_plots, i)
            i += 1
            plt.imshow(arr[b, 1], cmap='gray')           
            plt.colorbar(fraction=0.046, pad=0.04)
            plt.title(name + "_bg")
            
    #plt.subplot(1, n_plots, i)    
    #plt.imshow(loss_img[b, 0], cmap='Reds')           
    #plt.colorbar(fraction=0.046, pad=0.04)

    plt.show()

In [ ]:
mask_idx, mask_idx_small, mask_bits, _ = next(masks(1, scales=[1., 0.5]))

In [ ]:
b = 1024
def debug(gan, X, mask_idx, mask_idx_small):
    outs = gan._debug(X, mask_idx)
    return dict(zip(gan.debug_labels, outs))

real = X[b:b+64*6:6]
outs1 = debug(gan, real, mask_idx, mask_idx_small)
outs2 = debug(gan, real, np.ones_like(mask_idx)*MASK["IGNORE"], mask_idx_small)

In [ ]:
for i in range(64):
    #if outs["d_out"][i][0] > 0.5:
    draw_debug(mask_idx, mask_bits, outs1, i)

In [ ]:
mi = T.tensor4()
img = T.tensor4()
t_m_loss = theano.function([mi, img], mask_loss(mi, img))

In [ ]:
for layer in gan.gan.G.nodes.values():
    if hasattr(layer, 'params'):
        layer.build()
for layer in gan.gan.D.layers:
    if hasattr(layer, 'params'):
        layer.build()

In [ ]:
t_m_loss(mask_idx, outs["image"])

In [ ]:
loss_sum = 0


for b in range(64):
    loss = t_m_loss(mask_idx[b:b+1], outs["image"][b:b+1])
    print("{}: {}".format(b, loss))
    loss_sum += loss[0]
print(loss_sum / 64.)

In [ ]:
bwb_mask = bw_background_mask(theano.shared(mask_idx)).eval()
b = 34
name = "bwb_mask"
plt.imshow(bwb_mask[b, 0], cmap='gray')           
plt.title(name + "_bg") 
plt.show()
plt.imshow(bwb_mask[b, 1], cmap='gray')           
plt.title(name + "_bg")
plt.show()

In [ ]:
for i in range(len(outs["d_out"])):
    print("{}: {}".format(i, outs["d_out"][i][0]))

In [ ]:
for p in gan.gan.G.params:
    x = np.asarray(p.get_value())
    print(x[:])

In [ ]:
output_dir = "models/gan_dense_trained_#443/"
os.makedirs(output_dir, exist_ok=True)
gan.gan.save_weights(output_dir + "{}.hdf5")

In [ ]:
gan_load = MaskGAN()
gan_load.gan.load_weights(output_dir + "{}.hdf5")

In [ ]:
gan_load.compile(lambda: SGD(), functions=["train", "debug"])

In [ ]:
b = 512

mask_idx = next(masks(1))
outs = gan_load.debug(X[b:b+64], mask_idx)
print(outs.keys())

In [ ]:
m = theano.shared(mask_idx)
idx = T.gt(m, MASK["IGNORE"])
s = T.sum(idx, axis=[1, 2, 3]).eval()
print(s)
print(idx.shape.eval())
print(s / 64)

In [ ]:
x = np.ones((4, 1, 3, 3))
x[1] *= 2
T.mean(x, axis=[1, 2, 3]).eval()

In [ ]:
158621 / (64)